# Angry tweets
## Dominik Tomkiewicz 140793
## Wojciech Rzeczycki 140770

W ramach drugiego projektu z Zaawansowanej eksploracji danych przygotowano model predykcji klasyfikacji dla sentymentu tweetów.
Analizie poddany został zbiór danych kilku tysięcy tweetów.

# 1. Wykorzystane biblioteki

In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np
import csv
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# 2. Pobranie danych

Dane składają się z wpisów o tweetach: sentymentu użytkownika (pozytywna/negatywna/neutralna) oraz jego wpisu. Wszystkie słowa potratkowano jako stringi, dodatkowo ze zbioru danych testowych usunięto wartości puste występujące na końcu pliku.

Stworzono algorytm, który klasyfikuje tweety do odpowiedniego sentymentu.

In [2]:
sentiments = ['negative', 'positive', 'neutral']
stop_words = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 
              'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 
              'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 
              'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 
              'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 
              'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 
              'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 
              'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 
              'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 
              'doing', 'it', 'how', 'further', 'was', 'here', 'than']

tweets = pd.read_csv("train.csv", header=0, sep=',', dtype = str)

tmp = np.random.rand(len(tweets)) < 0.75
angryTweetsTrain = tweets[tmp]
angryTweetsTest = tweets[~tmp]

angryTweetsTrainData = []
angryTweetsTrainLabel = []

angryTweetsTestData = []
angryTweetsTestLabel = []

In [3]:
angryTweetsTrain.head(6)

,Id,Category,Tweet
0,635769805279248384,negative,Not Available
1,635930169241374720,neutral,IOS 9 App Transport Security. Mm need to check...
2,635950258682523648,neutral,"Mar if you have an iOS device, you should down..."
3,636030803433009153,negative,@jimmie_vanagon my phone does not run on lates...
4,636100906224848896,positive,Not sure how to start your publication on iOS?...
6,636276311560859648,neutral,If you're not already signed up to test my iOS...


In [4]:
angryTweetsTest.head(6)

,Id,Category,Tweet
5,636176272947744772,neutral,"Two Dollar Tuesday is here with Forklift 2, Qu..."
7,636302400546975744,neutral,"YouTube Gaming Officially Launches On Web, And..."
11,636484190050119681,positive,"Today @YouTubeGaming launches, with apps for i..."
14,636828460820729856,positive,#CrossSkyHigh is going IOS #saturday. For now ...
24,637333676099932160,neutral,Huawei Watch Pre-Order Suggests Android Wear C...
33,638206033639292932,positive,@tim_cook loving my new iPhone 6 from T-Mobile...


# 3. Przetwarzanie tweetów

Dla każdego wpisu w listach treningowej i testowej dokonujemy transformacji danych:

- usuwamy słowa zaczynające się od symbolu @ (służące do oznaczenia użytkownika, co nie dostarcza informacji na temat sentymentu),

- zamieniamy kody znaków na znaki,

- usuwamy słowa znajdujące się w zmiennej stop_words (słowa łączące wypowiedzi, nie dostarczające przydatnych informacji),

- po oczyszczeniu danych dodajemy do listy sentyment i zawartość tweeta (co posłuży jego klasyfikacji).

In [5]:
def replace(text):
    text = re.sub(r'@\w+', '', text)
    text = re.sub('&nbsp;', ' ', text)
    text = re.sub('&lt;', '<', text)
    text = re.sub('&gt;', '>', text)
    text = re.sub('&amp;', '&', text)
    text = re.sub('&pound;', u'£', text)
    text = re.sub('&euro;', u'€', text)
    text = re.sub('&copy;', u'©', text)
    text = re.sub('&reg;', u'®', text)
    return text

for index, tweet in angryTweetsTrain.iterrows():
    try:
        tweet[2] = replace(tweet[2])
        tweet_words = tweet[2].split()
        cleaned_words  = [word for word in tweet_words if word.lower() not in stop_words]
        tweet[2] = ' '.join(cleaned_words)
        angryTweetsTrainData.append(tweet[2])
        angryTweetsTrainLabel.append(tweet[1])
    except:
        print(type(tweet))
            
for index, tweet in angryTweetsTest.iterrows():
    try:
        tweet[2] = replace(tweet[2])
        tweet_words = tweet[2].split()
        cleaned_words  = [word for word in tweet_words if word.lower() not in stop_words]
        tweet[2] = ' '.join(cleaned_words)
        angryTweetsTestData.append(tweet[2])
        angryTweetsTestLabel.append(tweet[1])
    except:
            print(type(tweet))

# 4. Przypisanie wag słowom

Dla tweetów cechy, które pomogą w klasyfikacji to słowa, które w nich występują, a ich wagi to częstotliwość ich występowania w tweetach.

Za pomocą metody TfidfVectorizer transformujemy dane wejściowe w wektory dla klasyfikatora ustawiając odpowiednie parametry:

- min_df - odrzucane są słowa występujące w mniej niż określonej liczbie wpisów,

- max_df - odrzuca słowa pojawiające się w określonym ułamku dokumentów,

- sublinear_tf i use_idf - używanie sublinear weighting i wagowania IDF.

Za pomocą funkcji fit_transform zostanie stworzony słownik słowo-waga. Funkcja transform transformuje dane do wersji słownikowej.

In [6]:
vectorizer = TfidfVectorizer(min_df = 4,
                             max_df = 0.95,
                             sublinear_tf=True,
                             use_idf=True)
tweets_train_vectors = vectorizer.fit_transform(angryTweetsTrainData)
tweets_test_vectors = vectorizer.transform(angryTweetsTestData)

# 5. Trening

Po analizie możliwości klasyfikatorów dostarczonych przez Scikit-learn zdecydowano się na wykorzystanie algorytmu Naiwnego Bayesa. Poniżej kod trenujący klasyfikator.

In [7]:
classifier_NB = MultinomialNB()
classifier_NB.fit(tweets_train_vectors, angryTweetsTrainLabel)
prediction_NB=classifier_NB.predict(tweets_test_vectors)

In [8]:
print("Wyniki dla naiwnego Bayesa")
print(classification_report(angryTweetsTestLabel, prediction_NB))

Wyniki dla naiwnego Bayesa
              precision    recall  f1-score   support

    negative       0.56      0.06      0.11       236
     neutral       0.45      0.33      0.38       533
    positive       0.58      0.86      0.69       723

    accuracy                           0.54      1492
   macro avg       0.53      0.42      0.39      1492
weighted avg       0.53      0.54      0.49      1492



# 6. Testowanie

Poniżej kod testujący działanie klasyfikatora:

In [9]:
tweetsTrain = pd.read_csv("train.csv", header = 0, sep=',', dtype = str)
tweetsTest = pd.read_csv("test.csv", header = 0, sep=',', dtype = str).dropna()

tweetsTrainData = []
tweetsTrainLabel = []

tweetsTestId = []
tweetsTestData = []
tweetsTestLabel = []
            
for index, tweet in tweetsTrain.iterrows():
    try:
        tweet[2] = replace(tweet[2])
        tweet_words = tweet[2].split()
        cleaned_words  = [word for word in tweet_words if word.lower() not in stop_words]
        tweet[2] = ' '.join(cleaned_words)
        tweetsTrainData.append(tweet[2])
        tweetsTrainLabel.append(tweet[1])
    except:
            print(type(tweet))
    
for index, tweet in tweetsTest.iterrows():
    try:
        tweet[1] = replace(tweet[1])
        tweet_words = tweet[1].split()
        cleaned_words  = [word for word in tweet_words if word.lower() not in stop_words]
        tweet[1] = ' '.join(cleaned_words)
        tweetsTestData.append(tweet[1])
        tweetsTestId.append(tweet[0])
    except:
            print(type(tweet))
    
vectorizer = TfidfVectorizer(min_df = 4,
                             max_df = 0.95,
                             sublinear_tf=True,
                             use_idf=True)

train_vectors = vectorizer.fit_transform(tweetsTrainData)
test_vectors = vectorizer.transform(tweetsTestData)

classifier_NB_test = MultinomialNB()
classifier_NB_test.fit(train_vectors, tweetsTrainLabel)
prediction_NB_test=classifier_NB_test.predict(test_vectors)

header = {'Id': tweetsTestId, 'Category': prediction_NB_test}
result = pd.DataFrame(data = header)
result.to_csv('result.csv', sep = ',', columns=['Id', 'Category'], index=False)